# Generator step 1 - Initial data finding

[x] Function calling

[ ] Some way to manage facts - add results from tools calls, convert to XML, ...

[ ] Test if we have unfilled facts and or we got some new info in the last update (in previous task)

[ ] Add source info for each stored fact

[ ] Iteratively fill the fact sheet - convert from chain to graph

In [1]:
ASSET_REPORT = {
    'facts':{
        'today': 'Current date',
        'portfolio_today': 'List of holdings on specific day',
        'portfolio_last': 'List of holdings one month before report day'
    }
}

In [2]:
from langchain_core.tools import tool

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_google_vertexai import ChatVertexAI
from langchain_google_vertexai.model_garden import ChatAnthropicVertex

In [26]:
from datetime import date

@tool
def current_date() -> date:
    """
    Return today`s date
    """
    return date.today()

In [27]:
@tool
def get_customer_id(name: str) -> int:
    """
    Get customer id from the name
    """
    return 1

In [28]:
@tool
def holdings(customer: str, day: date):
    """
    Get list of holdings of client on specific day
    """
    if day.date >= 8:
        return [{'symbol': 'GOOGL',  'quantity': 100, 'cost_basis': 110}]       
    else:
        return [{'symbol': 'GOOGL',  'quantity': 90, 'cost_basis': 105}]       

In [29]:
holdings.args_schema.schema()

{'title': 'holdingsSchema',
 'description': 'Get list of holdings of client on specific day',
 'type': 'object',
 'properties': {'customer': {'title': 'Customer', 'type': 'string'},
  'day': {'title': 'Day', 'type': 'string', 'format': 'date'}},
 'required': ['customer', 'day']}

In [30]:
gemini = ChatVertexAI(model_name='gemini-1.5-pro-001')
sonnet = ChatAnthropicVertex(model='claude-3-5-sonnet@20240620', location='us-east5')

In [56]:
facts = '''<FACT><NAME>today</NAME><DESCRIPTION>Today`s date</DESCRIPTION></FACT>
<FACT><NAME>client_id</NAME><DESCRIPTION>Cliend ID</DESCRIPTION><FACT>
<FACT><NAME>portfolio_today</NAME><DESCRIPTION>List of holdings on specific day</DESCRIPTION></FACT>
'''

In [78]:
prompt = ChatPromptTemplate(
    messages=[
        ('system', 'You are an assistant collecting data for an report. Gather facts to fill in facts structure. Request only unknown facts'),
        ('user', 'Create asset report for {customer}\nKnown facts: {known}. All facts: {facts}')
    ]
)

In [79]:
chain = prompt | gemini.bind_tools([current_date, get_customer_id, holdings])

In [80]:
response = chain.invoke(input={'customer': 'Boris', 'facts' : facts, 'known': '<today>08/12/2024</today><customer_id>1</customer_id>'})

In [81]:
response.tool_calls

[{'name': 'holdings',
  'args': {'day': '08/12/2024', 'customer': '1'},
  'id': '8c8b3d7e-7213-4ac6-80ba-067024b3406e',
  'type': 'tool_call'}]

In [38]:
prompt.invoke({'customer': 'Boris', 'facts' : facts})

ChatPromptValue(messages=[SystemMessage(content='Gather facts to fill in facts structure'), HumanMessage(content='Create asset report for Boris\n<FACT><NAME>today</NAME><DESCRIPTION>Today`s date</DESCRIPTION></FACT><FACT>')])

str